In [4]:
import requests
import bs4
import re

import pandas as pd

In [5]:
# This will search on google for keywords and gives urls back
from googlesearch import search

city = "Montreal"
text_search = city +'attractions planetware'

links = []
def get_url(text = text_search):
    text1 = "tourist-attractions-/"
    text2 = city.lower()
    text3 = "planetware"
    for url in search(text_search, stop=10):
        links.append(url)
    for link in links:
        if text1 and text2 and text3 in link:
            return(link)

# TODO
# Moscow does not have an article yet it links to cz

In [6]:
# Gets the link
get_url()

'https://www.planetware.com/tourist-attractions-/montreal-cdn-qu-qum.htm'

In [7]:
# Save the link as a variable
url = get_url()

def get_http(url = url):
    res = requests.get(url)
    soup = bs4.BeautifulSoup(res.text,'lxml')
    attractions = soup.select('h2')
    
    att = [attraction.text for attraction in attractions]
    
    att_list = []
    
    # Remove the numbers and space in front of the text and save it in a list
    for attraction in att:
        #Need to fix, beginning missing number
        a = attraction.lstrip('0123456789.- ')
        att_list.append(a)
        #print(a)
    return att_list

In [8]:
# Gets the attractions for certain city
att_list = get_http()
att_list

['Mont-Royal',
 'Vieux-Montreal (Old Montreal)',
 'Jardin Botanique (Botanical Garden)',
 'Notre-Dame Basilica',
 "Oratoire Saint-Joseph (St. Joseph's Oratory)",
 'Parc Jean Drapeau',
 'Musée des Beaux Arts (Fine Arts Museum)',
 'Pointe-à-Callière',
 'Place des Arts',
 'Atwater and Jean Talon Markets',
 'St. Mary Queen of the World',
 'McCord Museum',
 'Square Saint-Louis and Rue Denis',
 'Lachine Canal National Historic Site',
 'Chinatown',
 'Where to Stay in Montreal for Sightseeing',
 'Tips and Tours: How to Make the Most of Your Visit to Montreal',
 'More on Canada']

The output of the function above shows that the last three elements in the list are irrelevant therefore we need to remove them.

In [9]:
# Based on research we know that the element containing 'Where to' and onwards so we can delete but first 
# need to find the index location of it

# Locate the index with the word "Where to"
index_of_sub = [i for i, s in enumerate(att_list) if 'Where to' in s]
index_of_sub = index_of_sub[0]

In [10]:
del att_list[index_of_sub: ]

In [11]:
att_list

['Mont-Royal',
 'Vieux-Montreal (Old Montreal)',
 'Jardin Botanique (Botanical Garden)',
 'Notre-Dame Basilica',
 "Oratoire Saint-Joseph (St. Joseph's Oratory)",
 'Parc Jean Drapeau',
 'Musée des Beaux Arts (Fine Arts Museum)',
 'Pointe-à-Callière',
 'Place des Arts',
 'Atwater and Jean Talon Markets',
 'St. Mary Queen of the World',
 'McCord Museum',
 'Square Saint-Louis and Rue Denis',
 'Lachine Canal National Historic Site',
 'Chinatown']

In [12]:
#df = pd.DataFrame(columns = ["Attraction"])

def to_df(list = att_list):
    df = pd.DataFrame()
    df['Attractions'] = att_list
    return df

In [13]:
df = to_df(att_list)
df

,Attractions
0,Mont-Royal
1,Vieux-Montreal (Old Montreal)
2,Jardin Botanique (Botanical Garden)
3,Notre-Dame Basilica
4,Oratoire Saint-Joseph (St. Joseph's Oratory)
5,Parc Jean Drapeau
6,Musée des Beaux Arts (Fine Arts Museum)
7,Pointe-à-Callière
8,Place des Arts
9,Atwater and Jean Talon Markets


## Geocoding to get coordinates

In [17]:
import googlemaps

list_of_lat = []
list_of_lon = []

city = 'Montreal'
country = 'Canada'

gmaps = googlemaps.Client(key='INSERT_YOUR_KEY_HERE')

def geocode_address(loc):
    query = loc + city + country
    geocode_result = gmaps.geocode(query)
    
    
    lat = geocode_result[0]["geometry"]["location"]["lat"]
    lon = geocode_result[0]["geometry"]["location"]["lng"]
    
    list_of_lat.append(lat)
    list_of_lon.append(lon)
    
for attraction in df['Attractions']:
    geocode_address(attraction)

df['Latitude'] = list_of_lat
df['Longitude'] = list_of_lon

In [18]:
df

,Attractions,Latitude,Longitude
0,Mont-Royal,45.507102,-73.587407
1,Vieux-Montreal (Old Montreal),45.507453,-73.554418
2,Jardin Botanique (Botanical Garden),45.560002,-73.563009
3,Notre-Dame Basilica,45.504525,-73.556120
4,Oratoire Saint-Joseph (St. Joseph's Oratory),45.491912,-73.616467
5,Parc Jean Drapeau,45.517066,-73.533580
6,Musée des Beaux Arts (Fine Arts Museum),45.498522,-73.579400
7,Pointe-à-Callière,45.502651,-73.554167
8,Place des Arts,45.508320,-73.566431
9,Atwater and Jean Talon Markets,45.479457,-73.576451
